In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import re

In [ ]:
import csv

In [46]:
df = pd.read_csv('zomato.csv')

In [47]:
df = pd.DataFrame(df)


In [48]:
df.shape

(51717, 17)

In [49]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [50]:
df.drop_duplicates(inplace=True)

In [51]:
df.isnull().sum()
df.dropna(how='any',inplace=True)

In [52]:
df = df.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city'})

In [53]:
df['cost'] = df['cost'].astype(str)
df['cost'] = df['cost'].apply(lambda x: x.replace(',','.'))
df['cost'] = df['cost'].astype(float)

In [54]:
df = df.loc[df.rate != 'NEW']
df = df.loc[df.rate != '-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x
df.rate = df.rate.apply(remove_slash).str.strip().astype('float')

In [55]:
df.name = df.name.apply(lambda x:x.title())
df.online_order.replace(('Yes','No'),(True, False),inplace=True)
df.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [56]:
restaurants = list(df['name'].unique())
df['Mean Rating'] = 0

for i in range(len(restaurants)):
    df['Mean Rating'][df['name'] == restaurants[i]] = df['rate'][df['name'] == restaurants[i]].mean()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
df[['Mean Rating']] = scaler.fit_transform(df[['Mean Rating']]).round(2)

In [57]:
df['reviews_list'] = df['reviews_list'].apply(lambda x: x.lower())

In [58]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
df['reviews_list'] = df['reviews_list'].apply(lambda text: remove_punctuation(text))
print(df['reviews_list'])

0        rated 40 ratedn  a beautiful place to dine int...
1        rated 40 ratedn  had been here for dinner with...
2        rated 30 ratedn  ambience is not that good eno...
3        rated 40 ratedn  great food and proper karnata...
4        rated 40 ratedn  very good restaurant in neigh...
                               ...                        
23041    rated 30 ratedn  nice place to hangout with ki...
23042    rated 40 ratedn  went there post dinner just f...
23043    rated 30 ratedn  place is good not that great ...
23044    rated 40 ratedn  a fine place to chill after o...
23045    rated 40 ratedn  nice and friendly place and s...
Name: reviews_list, Length: 23046, dtype: object


In [59]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df['reviews_list'] = df['reviews_list'].apply(lambda text: remove_stopwords(text))

In [61]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
df['reviews_list'] = df['reviews_list'].apply(lambda text: remove_urls(text))
df[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
1334,rated 40 ratedn taste 35nambiance 45nservice 4...,"South Indian, North Indian, Chinese"
14506,rated 30 ratedn went mandatory ice cream wisdo...,"Cafe, Desserts, Fast Food, Burger"
12349,rated 50 ratedn amazing place filled art must ...,"Cafe, Italian, Fast Food"
13611,rated 50 ratedn location 55nambience 55nfood 5...,"South Indian, North Indian, Chinese"
19079,rated 30 ratedn ordered chinese veg combo comb...,"Chettinad, South Indian, Biryani"


In [62]:
restaurants_names = list(df['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

df = df.drop(['address', 'rest_type', 'type', 'menu_item', 'votes', 'url'],axis=1, errors='ignore')

In [63]:
df_percent = df.sample(frac=0.5)
print(df_percent.shape)

(11523, 12)


In [64]:
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [66]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english') # Changed min_df to 1
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

cosin_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [77]:
# def recommend(name, cosine_similarities = cosin_similarities):
#     recommend_restaurant = []
#     idx = indices[indices == name].index[0]
#     score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False
#                                                                    )
#     top30_indexes = list(score_series.iloc[0:31].index)

#     for each in top30_indexes:
#         recommend_restaurant.append(list(df_percent.index)[each])

#     df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

#     for each in recommend_restaurant:
#         df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))

#     df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
#     df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

#     print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))

#     return df_new
# recommend('Pai Vihar')

In [67]:
df.head()

,name,online_order,book_table,rate,phone,location,dish_liked,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,True,True,4.1,080 42297555\r\n+91 9743772233,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,Banashankari,3.99
1,Spice Elephant,True,False,4.1,080 41714161,Banashankari,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,Banashankari,3.97
2,San Churro Cafe,True,False,3.8,+91 9663487993,Banashankari,"Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,Banashankari,3.58
3,Addhuri Udupi Bhojana,False,False,3.7,+91 9620009302,Banashankari,Masala Dosa,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,Banashankari,3.45
4,Grand Village,False,False,3.8,+91 8026612447\r\n+91 9901210005,Basavanagudi,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,Banashankari,3.58


In [68]:
df_copy = df.copy()

In [69]:
df.to_csv('dataset.csv', index=False)

In [70]:
df_copy = pd.read_csv('dataset.csv')

In [89]:
def recommend(name, cosine_similarities=cosin_similarities):
    recommend_restaurant = []

    # Check if the name exists in indices
    if name not in indices.values:
        print(f"{name} not found in the dataset!")
        return None

    try:
        # Get the index of the given restaurant
        idx = indices[indices == name].index[0]
    except IndexError:
        print(f"{name} not found in indices!")
        return None

    # Get similarity scores and sort them
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    top30_indexes = list(score_series.iloc[0:31].index)

    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])

    # Collect data for the recommended restaurants
    df_list = []
    for each in recommend_restaurant:
        # Select rows for the recommended restaurant
        df_sample = df_percent.loc[df_percent.index == each, ['cuisines', 'Mean Rating', 'cost']]
        if not df_sample.empty:
            df_sample = df_sample.sample()  # Randomly sample one row
            df_sample['name'] = each  # Add the 'name' column
            df_list.append(df_sample)

    if not df_list:
        print("No recommendations found!")
        return None

    # Combine into a single DataFrame
    df_new = pd.concat(df_list, ignore_index=True)

    # Remove duplicates
    df_new = df_new.drop_duplicates(subset=['name', 'cuisines', 'Mean Rating', 'cost'], keep='first')

    # Ensure top 10 restaurants
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

    # If less than 10 restaurants, fill with random choices
    if len(df_new) < 10:
        print("Fewer than 10 unique recommendations found. Filling with random options.")
        remaining_count = 10 - len(df_new)
        additional_restaurants = (
            df_percent[~df_percent.index.isin(df_new['name'])]
            .sample(n=remaining_count, replace=False)
            .reset_index()
        )
        additional_restaurants['name'] = additional_restaurants['index']
        additional_restaurants = additional_restaurants[['name', 'cuisines', 'Mean Rating', 'cost']]
        df_new = pd.concat([df_new, additional_restaurants], ignore_index=True)

    # Reorder columns and reset index
    df_new = df_new[['name', 'cuisines', 'Mean Rating', 'cost']]

    # Set index to start from 1
    df_new.index = range(1, len(df_new) + 1)

    print(f'TOP 10 RESTAURANTS LIKE {name} WITH SIMILAR REVIEWS:')
    return df_new

# Example call
recommend('Pai Vihar')

TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS:


,name,cuisines,Mean Rating,cost
1,Shree Thali,North Indian,3.94,150.0
2,Swad Punjab Da,North Indian,3.94,150.0
3,Cinnamon,"North Indian, Chinese, Biryani",3.71,550.0
4,Desi Doze,"North Indian, Fast Food",3.58,400.0
5,Food And You,"North Indian, Gujarati",3.58,300.0
6,Kakaji,North Indian,3.45,350.0
7,Prasiddhi Food Corner,"Fast Food, North Indian, South Indian",3.45,200.0
8,Mayura Sagar,"Chinese, North Indian, South Indian",3.32,250.0
9,A2B - Adyar Ananda Bhavan,"South Indian, North Indian, Chinese, Street Fo...",2.82,400.0
10,Punjabi Tasty Khana,"North Indian, Chinese, Biryani",2.68,450.0
